In [1]:
import nltk
from nltk.tag.stanford import StanfordNERTagger
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

In [2]:
from gensim.models.word2vec import LineSentence
from gensim.models import Phrases
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

#visualization libraries
import pyLDAvis
import pyLDAvis.gensim

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
import os
java_path = 'C:/Program Files/Java/jdk-10.0.1/bin/java.exe'
os.environ['JAVAHOME'] = java_path

import pandas as pd
import numpy as np
import codecs
import itertools as it
from bs4 import BeautifulSoup
import warnings
import pickle
from collections import Counter
import re

In [4]:
#import corpus/docket texts from html to pandas DataFrame
def grab_docket_test():
    files = []
    #get all .html files in the folder (all docket files are in .html)
    for file in os.listdir('docket_texts/test/'):
        if file.endswith('.html'):
            files.append(os.path.join('docket_texts/test/', file))

    df_docket_texts = pd.DataFrame()
    
    for i in range(len(files)): #gather all docket texts
    #for i in [0, 1]: #for testing purposes
        
        content = codecs.open(files[i], 'r', 'utf-8').read()
        #use beautiful soup to get the case ID
        soup = BeautifulSoup(content, 'lxml')
        case_id = str(soup.find_all('h3'))    
        bookmark1 = case_id.find('CASE #:') + len('CASE #:')
        bookmark2 = case_id.find('</h3>')
        case_id = case_id[bookmark1:bookmark2]

        #use pandas to grab tables in the html files
        docket_tables = pd.read_html(content)

        #error checking: gotta do this because there's different length of docket_list/
        #usually docket texts are in docket_list[3], but not always
        n = 0
        while docket_tables[n].isin(['Docket Text']).sum().sum() == 0:
            #print(n, docket_tables[n].isin(['Docket Text']).sum().sum())
            n += 1
                        
        #print(i, files[i])
        #print(docket_tables[n].head())

        #docket_tables[n] is the docket text table
        new_header = docket_tables[n].iloc[0]
        docket_tables[n] = docket_tables[n][1:]
        docket_tables[n].columns = new_header
        
        docket_tables[n]['#'] = pd.to_numeric(docket_tables[n]['#'],
                                              downcast = 'signed', errors = 'coerce')
        docket_tables[n]['Date Filed'] = pd.to_datetime(docket_tables[n]['Date Filed'])
        docket_tables[n]['Case ID'] = case_id

        df_docket_texts = pd.concat([df_docket_texts, docket_tables[n]])
    #reorder a column
    cols = list(df_docket_texts.columns)
    df_docket_texts = df_docket_texts[[cols[-1]] + cols[:-1]]
    
    print('current docket text table size/shape: {}'.format(df_docket_texts.shape))
    return df_docket_texts

In [5]:
%%time
df = grab_docket_test()
docket_original = list(df['Docket Text'])
for i in range(5):
    print('docket text {}'.format(i))
    print(docket_original[i], '\n')

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprec

current docket text table size/shape: (1804, 4)
docket text 0
FILING ERROR - DEFICIENT PLEADING - FILED AGAINST PARTY ERROR COMPLAINT against All Defendants. (Filing Fee $ 400.00, Receipt Number 0208-11793625)Document filed by Majid Soueidan.(Rosen, Samuel) Modified on 1/5/2016 (pc). (Entered: 01/04/2016) 

docket text 1
FILING ERROR PDF ERROR CIVIL COVER SHEET filed. (Rosen, Samuel) Modified on 1/5/2016 (pc). (Entered: 01/04/2016) 

docket text 2
REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-EASTERN CORPORATION, BRAD PEDERSEN, ROBERT J. KELLY, NELSON OBUS, WILLIAM M. SHOCKLEY, and SERGE DUPUIS, re: 1 Complaint. Document filed by Majid Soueidan. (Rosen, Samuel) (Entered: 01/04/2016) 

docket text 3
RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Corporate Parent. Document filed by Majid Soueidan.(Rosen, Samuel) (Entered: 01/04/2016) 

docket text 4
***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEADING. Notice to Attorney Samuel Kenneth Rosen to RE-FILE Document No. 1 Complaint,. The filing is 

In [6]:
%%time
path_to_model = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\classifiers\english.all.3class.distsim.crf.ser.gz'
path_to_jar = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\stanford-ner.jar'
tagger = StanfordNERTagger(path_to_model, path_to_jar = path_to_jar)

output = []
#length = 100 
length = len(docket_original)
for i in range(length):
    org_str = []
    name_str = []
    stripped_str1 = []
    stripped_str2 = []
    tokens = nltk.tokenize.word_tokenize(docket_original[i])
    for label in tagger.tag(tokens):
        #print(label)
        if label[1] == 'ORGANIZATION':
            org_str.append(label[0])
            stripped_str1.append('-ORG-')
        elif label[1] == 'PERSON':
            name_str.append(label[0])
            stripped_str1.append('-NAME-')
        else:
            stripped_str1.append(label[0])
            stripped_str2.append(label[0])
    
    output.append([docket_original[i],
                   ' '.join(org_str),
                   ' '.join(name_str),
                   ' '.join(stripped_str1),
                   ' '.join(stripped_str2)])

Wall time: 1h 54s


In [7]:
NER_df = pd.DataFrame(output, columns = ['Original Docket Text', 'Organization Portion', 'Name Portion', 
                                         'Identifying Org and Name', 'Stripped Org and Name'])

In [8]:
new_df = NER_df.copy()

In [9]:
print(new_df.head())
docket_text_list = list(new_df['Stripped Org and Name'])

                                Original Docket Text Organization Portion  \
0  FILING ERROR - DEFICIENT PLEADING - FILED AGAI...                        
1  FILING ERROR PDF ERROR CIVIL COVER SHEET filed...                        
2  REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...                        
3  RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...                        
4  ***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...                        

                                        Name Portion  \
0                        Majid Soueidan Rosen Samuel   
1                                       Rosen Samuel   
2  BRAD PEDERSEN ROBERT J. KELLY NELSON OBUS WILL...   
3                        Majid Soueidan Rosen Samuel   
4                               Samuel Kenneth Rosen   

                            Identifying Org and Name  \
0  FILING ERROR - DEFICIENT PLEADING - FILED AGAI...   
1  FILING ERROR PDF ERROR CIVIL COVER SHEET filed...   
2  REQUEST FOR ISSUANCE OF SUMMO

In [73]:
def text_preprocess1(text):
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    text = text.replace('-', '')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace('(s)', 's')
    text = text.replace("'s", 's')
    text = text.replace('*', '')
    text = text.replace('', '')
    text = text.replace('<', '')
    text = text.replace('>', '')
    text = text.replace('/', ' ')
    text = text.replace('\\', '')
    text = text.replace('&', ' ')
    return text

def text_preprocess2(text):
    text = text.replace('.', '')
    return text

def remove_stop(sentence):
    output = []
    for word in sentence.split():
        if word not in set(stopwords.words('english')):
            output.append(word)
    return ' '.join(output)

keywords = pd.read_csv('docket_texts/keywords.csv', header = None)
keywords.columns = ['keywords']
keyword_list = list(keywords['keywords'])

In [74]:
print(docket_text_list[1])
docket_text_list = [text_preprocess1(sentence).lower() for sentence in docket_text_list]
print(docket_text_list[1])
print(len(docket_text_list))

filing error pdf error civil cover sheet filed .  modified on 152016  . 
filing error pdf error civil cover sheet filed .  modified on 152016  . 
1804


In [75]:
class Splitter(object):

    def __init__(self):
        self.splitter = nltk.data.load('tokenizers/punkt/english.pickle')
        self.tokenizer = nltk.tokenize.TreebankWordTokenizer()

    def split(self,text):

        # split into single sentence
        sentences = self.splitter.tokenize(text)
        # tokenization in each sentences
        tokens = [self.tokenizer.tokenize(remove_stop(sent)) for sent in sentences]
        return tokens


class LemmatizationWithPOSTagger(object):
    def __init__(self):
        pass
    def get_wordnet_pos(self,treebank_tag):
        """
        return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
        """
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            # As default pos in lemmatization is Noun
            return wordnet.NOUN

    def pos_tag(self,tokens):
        # find the pos tagginf for each tokens [('What', 'WP'), ('can', 'MD'), ('I', 'PRP') ....
        pos_tokens = [nltk.pos_tag(token) for token in tokens]

        # lemmatization using pos tagg   
        # convert into feature set of [('What', 'What', ['WP']), ('can', 'can', ['MD']), ... ie [original WORD, Lemmatized word, POS tag]
        pos_tokens = [ [(word, lemmatizer.lemmatize(word,self.get_wordnet_pos(pos_tag)), [pos_tag]) for (word,pos_tag) in pos] for pos in pos_tokens]
        return pos_tokens

In [76]:
%%time
lemmatizer = WordNetLemmatizer()
splitter = Splitter()
lemmatization_using_pos_tagger = LemmatizationWithPOSTagger()

lemma_docket_text_list = []
for docket_text in docket_text_list:
    #step 1 split document into sentence followed by tokenization
    tokens = splitter.split(docket_text)

    #step 2 lemmatization using pos tagger 
    lemma_pos_token = lemmatization_using_pos_tagger.pos_tag(tokens)
    lemma_docket_text_list.append(lemma_pos_token)

Wall time: 1min 36s


In [77]:
print(len(lemma_docket_text_list)) #docket text document level
print(len(lemma_docket_text_list[0])) #docket text sentence level
print(len(lemma_docket_text_list[0][0])) #docket text word level
print(lemma_docket_text_list[0][0][0]) #docket text token level
print(lemma_docket_text_list[0][0][0][0]) #docket text tuple level

1804
3
10
('filing', 'file', ['VBG'])
filing


In [78]:
#let's do a collection of what we have
collection = {}
for lemma_pos_token in lemma_docket_text_list:
    for sentence in lemma_pos_token:
        for token in sentence:
            #print(token[2][0])
            if token[2][0] not in list(collection.keys()):
                collection[token[2][0]] = []
                collection[token[2][0]].append(token[1])
            else:
                if token[1] not in collection[token[2][0]]:
                    collection[token[2][0]].append(token[1])

In [ ]:
#pd.DataFrame(dict([ (k, pd.Series(v)) for k, v in collection.items()])).to_csv('NLP_pos.csv', index = False)

In [79]:
%%time
remove_pos = ["``", "NNPS", "NNP", "CD", '#', '$', "''", ",", "0", ":"]
remove_word = ["'s", "judge", "party", "defendant", "ex", "plantiff", "shall", "date", "b", "exhibit", "pennsylvania", "sign_judge", 
               "Inc..", "inc..", "llc", "'", "[_]", "action", "clerk", "july", "kw", "regard", "sac", "attachment", "c.d", "cal", "case", "cd", "l.p.", 
               "claim", "copy", "court", "direct", "form", "hereby", "magistrate", "p.c", "pl", "plaintiff", "regard", "sign", "time", "mr.", 
               "docket", "follow", "set", "matter" "agreement" "proceeding", "cotton", "january", "february", "march", "april", "may", "june", 
               "july", "august", "september", "october", "november", "december",
               "agreement", "v.", "place_vault", "modify", "fund", "associated", "provide", "material", "amount", "accordingly", "additional", 
               "second", "esq", "transmission", "g.c.", "seal", "review", "honor", "submit", "counsel", "witness", "civ", "first", "ltd..", "enter", 
               "stay", "forth", "matter", "whether", "class", "master", "information", "statement", "submission", "related", "see", "make", "paper", 
               "brookfield", "designate", "remain", "reportertranscriber", "submit", "include", "mail", "fact", "refer", "take", "pursuant", "amount", 
               "behalf", "I.p..", "must", "attorney",
               'abovecapitoned', 'attach', 'add', 'concern', 'chamber', 'close', 'district', 'damage', 'later', 
               'relate', 'return', 'require', 'restriction', 'respect', 'ny', 'seek', 'write', 'expert', 'transcript', 
               'day', 'h.o', 'damage', 'pre', 'proceeding', 'present', 'page', 'pending', 'p.m.', 'frcp', 'g.c.', 'record', 'r.']

    
#rebuild corpus
docket_texts_output = [] #ultimate output after cleaning

for lemma_pos_token in lemma_docket_text_list:
    docket_text_output = [] 
    for sentence in lemma_pos_token:
        sentence_output = []
        for token in sentence:
            #print(token[1])
            
            if token[2][0] not in remove_pos: #if the pos is not in the remove_pos list
                if token[1] not in remove_word: #these are the intentionally left out words
                    sentence_output.append(token[1]) #append the the sentence
        docket_text_output.append(' '.join(sentence_output))
    docket_texts_output.append(docket_text_output)
print(docket_texts_output[:10])

[['file error deficient plead file error complaint .', 'document file .', '.'], ['file error pdf error civil cover sheet file .', '.'], ['request issuance summons breezeeastern corporation complaint .', 'document file .'], ['rule corporate disclosure .', 'corporate parent .', 'document file .'], ['notice deficient pleading .', 'notice refile document .', 'complaint .', 'file deficient reason radio button select .', 'refile plead use event type complaint find event list complaint initiate document correct pdf select individually name filerfilers select individually name partyparties plead .'], ['notice modification .', 'notice .', 'partyparties .', 'partyparties reasonreasons name contain typographical error text omit .'], ['notice deficient civil cover sheet .', 'notice refile document .', 'civil cover sheet .', 'file deficient reason citizenship principal fill despite federal question jurisdiction .'], ['electronic summons issue .'], ['notice open statistical error correction notice .

In [80]:
new_df['Removed unnecessary POS & vocab'] = pd.Series(docket_texts_output)
new_df.head()

,Original Docket Text,Organization Portion,Name Portion,Identifying Org and Name,Stripped Org and Name,Removed unnecessary POS & vocab,Apply Trigram Phrase Model
0,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,,Majid Soueidan Rosen Samuel,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,[file error deficient plead file error complai...,[file error_deficient plead file error complai...
1,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,,Rosen Samuel,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,[file error pdf error civil cover sheet file ....,[file error pdf error civil_cover_sheet file ....
2,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,,BRAD PEDERSEN ROBERT J. KELLY NELSON OBUS WILL...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,[request issuance summons breezeeastern corpor...,[request_issuance_summons breezeeastern corpor...
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,,Majid Soueidan Rosen Samuel,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...,"[rule corporate disclosure ., corporate parent...","[., ., document file .]"
4,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,,Samuel Kenneth Rosen,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,"[notice deficient pleading ., notice refile do...","[notice deficient pleading ., notice refile do..."


In [81]:
def trigram_transform(texts):
    trigram_output = []

    remove_trigram = ['calendar_day', 'court_notice_intend', 'minute_entry_proceeding_hold', 'court_reportertranscriber_abovecaptioned_matter',
                      'redaction_calendar_day', 'rule_statement', 'obtain_pacer', 'may_obtain_pacer', 'reportertranscriber_abovecaptioned_matter',
                      'redact_transcript_deadline', 'send_chamber', "official_transcript_notice_give", "notice_intent_request", "proceed_hold", 
                      "fee_receipt_number", "civil_procedure", "pursuant_frcp", "official_transcript_conference", 
                      "purchase_reportertranscriber_deadline_release", "et_al", "mail_chamber", "transcript_restriction", "redaction_transcript", 
                      "transcript_view_public_terminal", "transcript_make_remotely", "associated_et_al", "electronically_available_public_without", 
                      "genesys_id", "release_transcript_restriction", "adar_bay", "redaction_request_due", "new_york", "official_transcript_conference", 
                      "transcript_make_remotely", "transcript_proceeding_conference_hold", "redaction_transcript",
                      'affidavit_jr._c.p.a', 'corporate_parent', 'certain_underwriter', 'federal_rule_civil_procedure', 'redaction_request', 
                      'official_transcript', 'rule_disclosure', 'rule_corporate_disclosure', 'place_vault', 'public_without_redaction_calendar', 
                      'purchase_deadline_release_transcript', 'transcript_proceeding_hold', 'transcript_remotely_electronically_available']
  

    for sentence in texts:
        unigram_review = []
        for word in sentence.split():
            unigram_review.append(word)
    
        #print('Uni: ', unigram_review)
        bigram_review = bigram_model[unigram_review]
        #print('Bi: ', bigram_review)
        trigram_review = trigram_model[bigram_review]
        trigram_review = [phrase for phrase in trigram_review if phrase not in remove_trigram]
        #print('Tri: ', trigram_review)
        trigram_output.append(' '.join(trigram_review))
    return trigram_output

In [82]:
bigram_model_filepath = 'docket_texts/bigram_model_noorgnoname'
trigram_model_filepath = 'docket_texts/trigram_model_nonamenoorg'

bigram_model = Phrases.load(bigram_model_filepath)
trigram_model = Phrases.load(trigram_model_filepath)

new_df['Apply Trigram Phrase Model'] = new_df['Removed unnecessary POS & vocab'].apply(trigram_transform)
new_df['Apply Trigram Phrase Model'].iloc[2]

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['request_issuance_summons breezeeastern corporation complaint .',
 'document file .']

In [83]:
#write trigram to file, not sure why...
trigram_dockets_filepath = 'docket_texts/test/trigram_transformed_dockets_noorgnoname.txt'

with codecs.open(trigram_dockets_filepath, 'w', encoding= 'utf_8') as f:
    for i in range(len(new_df['Apply Trigram Phrase Model'])):
        f.write(' '.join(new_df['Apply Trigram Phrase Model'][i]) + '\n')

In [84]:
trigram_dictionary_filepath = 'docket_texts/test/trigram_dict_noorgnoname.dict'

In [85]:
%%time

#some dictionary hyperparameters:
no_below = 10 #reference is 10
no_above = 0.4 #reference is 0.4

trigram_reviews = LineSentence(trigram_dockets_filepath)

# learn the dictionary by iterating over all of the reviews
trigram_dictionary = Dictionary(trigram_reviews)

# filter tokens that are very rare otrigram_reviewsr too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below = no_below, no_above = no_above) #this step is questionable. May need to change the parameters
trigram_dictionary.compactify()

trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
#trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

Wall time: 54.1 ms


In [86]:
trigram_bow_filepath = 'docket_texts/test/trigram_bow_corpus_noorgnoname.mm'

In [87]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        #print(review)
        #print(trigram_dictionary.doc2bow(review))
        yield trigram_dictionary.doc2bow(review)

In [88]:
trigram_sentences_filepath = 'docket_texts/test/trigram_sentences_nonamenoorg.txt'

with codecs.open(trigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for trigram_sentence in list(new_df['Apply Trigram Phrase Model']):
        #print(' '.join(trigram_sentence))
        f.write(' '.join(trigram_sentence) + '\n')
        
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [89]:
trigram_bow_filepath = 'docket_texts/test/trigram_bow_corpus_noorgnoname.mm'
trigram_dictionary_filepath = 'docket_texts/test/trigram_dict_noorgnoname.dict'

In [90]:
%%time

# generate bag-of-words representations for
# all reviews and save them as a matrix
MmCorpus.serialize(trigram_bow_filepath, trigram_bow_generator(trigram_sentences_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)
print(trigram_bow_corpus)

MmCorpus(1804 documents, 319 features, 9389 non-zero entries)
Wall time: 80.7 ms


In [91]:
%%time

#some dictionary hyperparameters:
no_below = 10 #reference is 10
no_above = 0.4 #reference is 0.4

trigram_reviews = LineSentence(trigram_dockets_filepath)

# learn the dictionary by iterating over all of the reviews
trigram_dictionary = Dictionary(trigram_reviews)

# filter tokens that are very rare otrigram_reviewsr too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below = no_below, no_above = no_above) #this step is questionable. May need to change the parameters
trigram_dictionary.compactify()

trigram_dictionary.save(trigram_dictionary_filepath)

Wall time: 76.7 ms


In [92]:
lda = LdaMulticore.load('docket_texts/train/lda_model_noorgnomodel_4')
train_dict = Dictionary.load('docket_texts/train/trigram_dict_noorgnoname.dict')

test_dict = Dictionary.load(trigram_dictionary_filepath)

In [93]:
for new_id, token in test_dict.iteritems():
    print(new_id, token)

231 pak6
13 entry
4 correct
44 error_deficient_entry
56 pro_hac_vice
137 call
143 setreset_deadline
78 support
73 oral_argument
30 address
180 without_prejudice
171 reason_state
132 complaint..
12 due
125 join
10 refile
163 approval
271 cost
160 memo_endorsement
269 escrow
281 reimbursement
21 order
155 either
133 deem
186 summons_issue
151 compel_arbitration
311 donee
62 brief
16 notice_appearance
152 status_conference_hold
314 administration
233 j
52 standing
170 withdraw
223 endorsed_letter_address
39 adjourn_conference
140 minute_entry_hold_telephone
181 also
305 asset
273 petition
302 temporary
232 agent
50 propose
130 receive
277 thereof
95 transmit
312 estate
210 dated
263 official
279 expense
254 law
103 new
41 adjourn
61 serve_answer_due
114 extend
168 proceed
248 associate
204 office
149 application_grant
85 calendar
182 hear
23 rule
90 without_redaction_calendar
193 counterclaim
141 nomorerack.com
58 via
32 joint
131 amendcorrect
9 notice
303 testimony
60 affidavit_service
2

In [106]:
topic_names = {0: 'motion to dismiss',
               1: 'motion for summary judgment',
               2: 'Complaint and motion',
               3: 'Amended Complaint and motion'}

topic_assignments = lda[trigram_bow_corpus]

model_output = []

for topic_assignment in topic_assignments:
    model_output.append([(topic_names[topic[0]], topic[1]) for topic in sorted(topic_assignment, key = lambda x: x[1], reverse = True)])

In [108]:
new_df['Topic Output'] = model_output

In [110]:
print(new_df.shape)
new_df.head()

(1804, 8)


,Original Docket Text,Organization Portion,Name Portion,Identifying Org and Name,Stripped Org and Name,Removed unnecessary POS & vocab,Apply Trigram Phrase Model,Topic Output
0,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,,Majid Soueidan Rosen Samuel,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,[file error deficient plead file error complai...,[file error_deficient plead file error complai...,"[(motion for summary judgment, 0.807523), (Ame..."
1,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,,Rosen Samuel,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,[file error pdf error civil cover sheet file ....,[file error pdf error civil_cover_sheet file ....,"[(Amended Complaint and motion, 0.73956877), (..."
2,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,,BRAD PEDERSEN ROBERT J. KELLY NELSON OBUS WILL...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,[request issuance summons breezeeastern corpor...,[request_issuance_summons breezeeastern corpor...,"[(motion for summary judgment, 0.74496585), (C..."
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,,Majid Soueidan Rosen Samuel,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...,"[rule corporate disclosure ., corporate parent...","[., ., document file .]","[(motion to dismiss, 0.25), (motion for summar..."
4,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,,Samuel Kenneth Rosen,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,"[notice deficient pleading ., notice refile do...","[notice deficient pleading ., notice refile do...","[(motion to dismiss, 0.561045), (motion for su..."


In [114]:
new_df[['Original Docket Text','Apply Trigram Phrase Model', 'Topic Output']].to_csv('docket_texts/test/verify_this.csv', index = False)